**IELE_4014**  
**Felipe Velásquez Montoya**
&nbsp;&nbsp;&nbsp;&nbsp; *cód estudiante:* 201632422
# Reto 3
Problema de regresión lineal para predecir la calidad del vino (https://archive.ics.uci.edu/ml/datasets/Wine+Quality) utilizando desarrollo propio.

**Paso 0**   Instalanción de librería numpy para facilitar operaciones con matrices

In [1]:
%pip install --user numpy

Note: you may need to restart the kernel to use updated packages.


Importación de dependencias random, math y numpy.

In [2]:
#Se importa numpy para facilitar las operaciones con matrices
import random
import math
import numpy as np

# Desarrollo de la regresión lineal

Se muestra, a continuación, el código de la regresión lineal y sus respectivas funciones auxiliares. se inicia con la función *train_test_split*, diseñada para imitar el comportamiento de su homónima en sklearn. 

Este método divide un conjunto de datos en datos de entrenamiento y datos de prueba. Lo anterior se hace de manera aleatoria, por lo que no hay garantía alguna de que los conjuntos resultantes tengan la misma distribución de probabilidad.

La metodología utilizada consiste en recorrer las listas y generar un número al azar por cada elemento, si el número generado es menor o igual a la rata dada por el tamaño del conjunto de entrenamiento dividido entre el tamaño total del conjunto, se añade al conjunto de entrenamiento. De modo contrario, se añade al conjunto de pruebas.

In [3]:
#Método auxiliar que divide los datos entre entranamiento y test
#test_ratio es el porcentaje total de los datos que se usará para test.
#Hace la división al azar utilizando random.random
#Pre: X_set  y y_set deben tener el mismo orden, es decir, para todo elemento X_i 
# de la X_set, y_i en el conjunto y_set debe ser su respectiva pareja.
def train_test_split(X_set, y_set, test_ratio):
    
    test_size = math.floor(test_ratio*len(X_set))
    current_test_size = 0
        
    X_train = []
    y_train = []
    X_test = []
    y_test = []
    
    for i in range(len(X_set)):
        
        if random.random() <= test_ratio and current_test_size < test_size:
            X_test.append(X_set[i])
            y_test.append(y_set[i])
            current_test_size += 1
        else:
            X_train.append(X_set[i])
            y_train.append(y_set[i])
    
    #En caso de que test no quede del tamaño requerido, se sacan los últimos elementos de train
    while len(X_test) != test_size:
        X_test.append(X_train.pop())
        y_test.append(y_train.pop())
    
    return X_train, X_test, y_train, y_test
            

Se presenta, a continuación, la clase LinearRegression. Esta contiene la función *fit* que utiliza un descenso de gradiente estocástico para entrenar el modelo. La función *residual_sum_of_squares* sirve de función auxiliar y la función *rsquared* sirve para la evaluación.

*rsquared* calcula el r cuadrado de la regresión.   
*residual_sum_of_squares* calcula la suma de cuadrados de los residuos.

In [4]:
class LinearRegression:
    
    def __init__(self, suppress_output = False):
        self.W = None
        self.suppress_output = suppress_output

    #Descenso estocástico de gradiente para regresión lineal.
    #training_data_X es una matriz que representa el conjunto de x_i's
    #training_data_y es una matriz que representa el conjunto de y_i's. Esta matriz tiene únicamente una 
    #Pre: training_data_X  y training_data_y deben tener el mismo orden, es decir, para todo elemento X_i 
    # de la matriz training_data_X, y_i en la matriz training_data_y debe ser su respectiva pareja.
    # ejemplo, la fila 0 de training_data_X tiene como respectivo valor y el que está dado por 
    # la fila 0 de training_data_y
    def fit(self, training_data_X, training_data_y,learning_rate= 0.0001, max_iterations = 10000):
    
        #Para evitar que la embarre y ponga un learning rate demasiado alto que lleve a que el método no converja 
        if(learning_rate > 0.0001):
            learning_rate = 0.0001
    
        #Se inicializa W en ceros, de dimensión (columnas X, 1).
        #Se hace así para que no se tenga que transponer W.
        W = np.zeros(shape = (1, len(training_data_X[0])))
    
        iterations = 0
        no_improv_it = 0
        last_error = 0
        new_error = 0
    
        #Se estimará el gradiente a partir, únicamente, de uno de los datos
        while iterations < max_iterations and no_improv_it < 100 :
        
            #Seleccionar un dato al azar.
            i = math.floor(random.random()*len(training_data_X))
    
            #Calcular error del dato seleccionado al azar.
            e = np.dot(training_data_X[i], W.transpose())[0] - training_data_y[i]
            #Estimar gradiente con el error anterior
            estimated_grad = np.multiply(e, training_data_X[i])
        
            W = W - np.multiply(learning_rate, estimated_grad)
            
            if last_error <= e:
                no_improv_it += 1
            elif last_error > e:
                no_improv_it = 0
        
            last_error = e
            iterations += 1        
        
        self.W = W
        
        if not self.suppress_output:
            print("----------------------------------------------")
            print("W found: %s" % W)
            print("R^2 found for the model: %s" % self.rsquare(training_data_X, training_data_y))
           
    def rsquare(self, test_data_X, test_data_y):
    
        residual_sum = self.residual_sum_of_squares(test_data_X, test_data_y)
    
        mean = np.mean(test_data_y)
        
        total_sum_squares = 0
        for i in range(0, len(test_data_y)):
            total_sum_squares +=  math.pow(test_data_y[i] - mean,2)
    
        return 1 - residual_sum/total_sum_squares
     
    #Calcula el error de la función, se usa sólo una vez por iteración.
    def residual_sum_of_squares(self, training_data_X, training_data_y):
    
        sum = 0
        for i in range(0, len(training_data_X)):
            sum+= math.pow(training_data_y[i] - np.dot(training_data_X[i] , self.W.transpose())[0],2)
    
        return 0.5 * sum

# Solución del problema de regresión lineal
Ya expuesta la regresión lineal que se desarrolló, se probará la misma atacando el problema de estimación de calidad del vino blanco.   
Se inicia cargando los datos del CSV:

In [5]:
data_matrix = np.loadtxt(open("./WineQuality/winequality-white.csv", "rb"), delimiter=";", skiprows=1)

print("Filas de la matriz: " + str(len(data_matrix)))
print("Columnas de la matriz: " + str(len(data_matrix[0])))

X = np.resize(data_matrix, (len(data_matrix), len(data_matrix[0])-1))
y = data_matrix[:,len(data_matrix[0])-1]

Filas de la matriz: 4898
Columnas de la matriz: 12


A continuación, se utiliza el método train_test_split para obtener el los conjuntos de prueba y de entrenamiento.

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_ratio = 2000.00/float(len(X))) 

print("Tamaño X entrenamiento: %s | Tamaño y entrenamiento: %s" % (len(X_train), len(y_train)))
print("Tamaño X prueba: %s | Tamaño y prueba: %s" % (len(X_test), len(y_test)))

Tamaño X entrenamiento: 2898 | Tamaño y entrenamiento: 2898
Tamaño X prueba: 2000 | Tamaño y prueba: 2000


El conjunto de entrenamiento se vuelve a partir, esta vez en conjuntos de 100, 1000 y 2898 datos. Como se constata por la salida de consola, ahora se tienen tres conjuntos de entrenamiento.

In [7]:
sizes = [100.00, 1000.00, 2898.00]
X_train_array = []
y_train_array = []

for i in sizes:
    Xy = train_test_split(X_train, y_train, test_ratio = 1.00 - i/float(len(X_train)))
    X_train_array.append(Xy[0])
    y_train_array.append(Xy[2])
    
#Verificar tamaños de los nuevos sets de entrenamiento
for i in range(0, len(X_train_array)):
    print("-----------------------------")
    print("Set de entrenamiento %s" % i)
    print("    Tamaño X de entrenamiento: %s | Tamaño y de entrenamiento: %s" 
          % (len(X_train_array[i]), len(y_train_array[i])))

-----------------------------
Set de entrenamiento 0
    Tamaño X de entrenamiento: 100 | Tamaño y de entrenamiento: 100
-----------------------------
Set de entrenamiento 1
    Tamaño X de entrenamiento: 1000 | Tamaño y de entrenamiento: 1000
-----------------------------
Set de entrenamiento 2
    Tamaño X de entrenamiento: 2898 | Tamaño y de entrenamiento: 2898


A continuación, se entrenan tres regresiones lineales, una con cada conjunto de entrenamiento y se prueba su desempeño con el conjunto de datos de prueba. Como podrá observarse, el desempeño obtenido por el modelo es extremadamente pobre en todos los casos, sugiriendo que el problema no puede ser resuelto por medio de una regresión lineal. Debe resaltarse que dado que no se reservaron datos de validación, los modelos no son comparables.

In [8]:
linreg_array = []
linreg_score = []

for i in range(0, len(X_train_array)):
    #Se entrena la regresión lineal con el conjunto de entrenamiento correspondiente.
    lr = LinearRegression(suppress_output = True)
    lr.fit(X_train_array[i], y_train_array[i], learning_rate = 0.000001)
    linreg_array.append(lr)
    
    #Se prueba la regresión lineal con los conjuntos de prueba:
    linreg_score.append(linreg_array[i].rsquare(X_test, y_test))
    
    #Se imprimen los resultados del modelo
    print("El Modelo entrenado con %s datos obtuvo un R cuadrado de %s" % (sizes[i], linreg_score[i]))

El Modelo entrenado con 100.0 datos obtuvo un R cuadrado de -1.8782959123186918
El Modelo entrenado con 1000.0 datos obtuvo un R cuadrado de -1.7596193294154268
El Modelo entrenado con 2898.0 datos obtuvo un R cuadrado de -1.7425412061483234


En vista del pobre desempeño obtenido se intentará entrenar la regresión con un conjunto de datos estandarizados.   
Se importa sklearn para utilizar su función MinMaxScaler de estandarización.

In [9]:
%pip install sklearn

Note: you may need to restart the kernel to use updated packages.


Como puede comprobarse a continuación, los resultados obtenidos estandarizando los datos fueron aún más pobres que con los datos crudos. De nuevo, sugiriendo que el problema no se puede resolver por medio de un modelo lineal.

In [10]:
from sklearn.preprocessing import MinMaxScaler

data_matrix_norm = np.loadtxt(open("./WineQuality/winequality-white.csv", "rb"), delimiter=";", skiprows=1)

print("Filas de la matriz: " + str(len(data_matrix_norm)))
print("Columnas de la matriz: " + str(len(data_matrix_norm[0])))

X_norm = np.resize(data_matrix_norm, (len(data_matrix_norm), len(data_matrix_norm[0])-1))
y_norm = data_matrix[:,len(data_matrix[0])-1]

X_norm = MinMaxScaler().fit_transform(X_norm)

y_norm= np.array([y_norm]).transpose()
y_norm = MinMaxScaler().fit_transform(y_norm)

X_train_norm, X_test_norm, y_train_norm, y_test_norm = train_test_split(X_norm, y_norm, test_ratio = 2000.00/float(len(X))) 

print("Tamaño X entrenamiento: %s | Tamaño y entrenamiento: %s" % (len(X_train_norm), len(y_train_norm)))
print("Tamaño X prueba: %s | Tamaño y prueba: %s" % (len(X_test_norm), len(y_test_norm)))

sizes_norm = [100.00, 1000.00, 2898.00]
X_train_array_norm = []
y_train_array_norm = []

for i in sizes_norm:
    Xy_norm = train_test_split(X_train_norm, y_train_norm, test_ratio = 1.00 - i/float(len(X_train_norm)))
    X_train_array_norm.append(Xy_norm[0])
    y_train_array_norm.append(Xy_norm[2])
    
#Verificar tamaños de los nuevos sets de entrenamiento
for i in range(0, len(X_train_array_norm)):
    print("-----------------------------")
    print("Set de entrenamiento %s" % i)
    print("    Tamaño X de entrenamiento: %s | Tamaño y de entrenamiento: %s" 
          % (len(X_train_array_norm[i]), len(y_train_array_norm[i])))
    
linreg_array_norm = []
linreg_score_norm = []

for i in range(0, len(X_train_array)):
    #Se entrena la regresión logística con el conjunto de entrenamiento correspondiente.
    lr = LinearRegression(suppress_output = True)
    lr.fit(X_train_array_norm[i], y_train_array_norm[i])
    linreg_array_norm.append(lr)
    
    #Se prueba la regresión logística con los conjuntos de prueba:
    linreg_score_norm.append(linreg_array_norm[i].rsquare(X_test_norm, y_test_norm))
    
    #Se imprimen los resultados del modelo
    print("El Modelo entrenado con %s datos obtuvo un R cuadrado de %s" % (sizes_norm[i], linreg_score_norm[i]))

Filas de la matriz: 4898
Columnas de la matriz: 12
Tamaño X entrenamiento: 2898 | Tamaño y entrenamiento: 2898
Tamaño X prueba: 2000 | Tamaño y prueba: 2000
-----------------------------
Set de entrenamiento 0
    Tamaño X de entrenamiento: 100 | Tamaño y de entrenamiento: 100
-----------------------------
Set de entrenamiento 1
    Tamaño X de entrenamiento: 1000 | Tamaño y de entrenamiento: 1000
-----------------------------
Set de entrenamiento 2
    Tamaño X de entrenamiento: 2898 | Tamaño y de entrenamiento: 2898
El Modelo entrenado con 100.0 datos obtuvo un R cuadrado de -4.43474729859357
El Modelo entrenado con 1000.0 datos obtuvo un R cuadrado de -4.436704686590301
El Modelo entrenado con 2898.0 datos obtuvo un R cuadrado de -4.426233465500838


Para probar que la regresión lineal desarrollada sirve correctamente, se hará un modelo sobre un conjunto de datos sintético diseñado para regresiones lineales. Para realizar esto se utilizará la librería mglearn para generar el conjunto de datos, adicionalmente, se comparará el desempeño de la regresión lineal desarrollada con la de los modelos de sklearn.

In [11]:
%pip install --user mglearn

Note: you may need to restart the kernel to use updated packages.


In [15]:
import mglearn
X_synth, y_synth = mglearn.datasets.make_wave(n_samples=5000)

X_train_synth, X_test_synth, y_train_synth, y_test_synth = train_test_split(X_synth, y_synth, test_ratio = 0.3*5000/float(len(X))) 

lr_synthetic = LinearRegression(suppress_output = True)
lr_synthetic.fit(X_train_synth, y_train_synth)
print("R cuadrado para los datos sintéticos utilizando el modelo propio: %s" % lr_synthetic.rsquare(X_test_synth, y_test_synth))

R cuadrado para los datos sintéticos utilizando el modelo propio: 0.8085714114452791


In [14]:
from sklearn.linear_model import LinearRegression as sklr

print("R cuadrado para los datos sintéticos utilizando sklearn: %s " % sklr().fit(X_train_synth, y_train_synth).score(X_test_synth,y_test_synth))

R cuadrado para los datos sintéticos utilizando sklearn: 0.6397809665821652 


Como puede observarse, al aplicar la regresión lineal desarrollada a un conjunto de datos sintético se obtiene un resultado con un R cuadrado alto y comparable con el obtenido si se utiliza la librería sklearn.